In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from keras import layers
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Dropout, Activation,Flatten
from keras import metrics, regularizers
from tensorflow.keras.optimizers import RMSprop
from sklearn import metrics
from keras.layers import Dense, SimpleRNN

In [ ]:
ddf = pd.read_csv("/content/yuz_veri_kumesi.csv", sep=";")
df = ddf[['Cumle','Sinif']]

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 50
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Cumle'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df['Cumle'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['Sinif']).values
print('Shape of label tensor:', Y.shape)

Found 20646 unique tokens.
Shape of data tensor: (6109, 50)
Shape of label tensor: (6109, 7)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.7, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1832, 50) (1832, 7)
(4277, 50) (4277, 7)


In [ ]:
epochs = 30
batch_size = 256
model = Sequential()
model.add(Flatten())
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(256, activation='relu', dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

model.add(LSTM(128, activation='relu', dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128, activation='relu', dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

model.add(LSTM(16, activation='relu', dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(512, activation='LeakyReLU'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

accr= model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.4f}\n  Accuracy: {:0.4f}'.format(accr[0],accr[1]))

Epoch 1/30
7/7 [==============================] - 24s 1s/step - loss: 1.9152 - acc: 0.4624 - val_loss: 1.8472 - val_acc: 0.5380
Epoch 2/30
7/7 [==============================] - 8s 1s/step - loss: 8.2800 - acc: 0.5370 - val_loss: 1.4430 - val_acc: 0.5380
Epoch 3/30
7/7 [==============================] - 8s 1s/step - loss: 1.5874 - acc: 0.5370 - val_loss: 1.6718 - val_acc: 0.5380
Epoch 4/30
7/7 [==============================] - 8s 1s/step - loss: 1.6448 - acc: 0.5370 - val_loss: 1.6187 - val_acc: 0.5380
Epoch 5/30
7/7 [==============================] - 8s 1s/step - loss: 1.5733 - acc: 0.5370 - val_loss: 1.5248 - val_acc: 0.5380
Epoch 6/30
7/7 [==============================] - 8s 1s/step - loss: 1.4591 - acc: 0.5370 - val_loss: 1.3710 - val_acc: 0.5380
Epoch 7/30
7/7 [==============================] - 8s 1s/step - loss: 1.2703 - acc: 0.5370 - val_loss: 1.2608 - val_acc: 0.5380
Epoch 8/30
7/7 [==============================] - 8s 1s/step - loss: 1.2297 - acc: 0.5370 - val_loss: 1.2456 -

In [ ]:
y_test = [row[0] for row in Y_test]
from sklearn import metrics
y_pred=model.predict(X_test)
dene = np.argmax(y_pred, axis=1)
a = np.argmax(Y_test, axis=1)
print("F1: ",f1_score(a[:],dene[:],average='weighted'))


F1:  0.4868011578207406
